In [1]:
import sys
sys.path.append('f:/Project/PcAnalyse/')

import datetime
import tensorflow as tf
import StockDataLoad as sdl
import StockLearningData as sld
import StockBackTest as sbt

In [2]:
def get_predict_data(k, date):
#     k = ts.get_hist_data(c, start=start_date, end=end_date)

    if k is None or (sld.SLD_HIST_PATTERN_LEN + 5) > len(k):
#         print(code + " too small.")
        return None
        
    #k = k.iloc[::-1]

#     sld.preliminary_data(k)

    try:
        idx = k.index.get_loc(date)
    except KeyError:
#         print("{} {} not found.".format(code, date))
        return None
    
    k_predict = sld.get_predict_item(k, idx)
    
    return k_predict


In [3]:
import numpy as np
import pandas as pd
# date = '2017/08/18'
# code_list = ts.get_gem_classified()

def get_predicted_df(classifier, stock_dict, date):
    predicted_list = []
    code_list = []

    for c, v in stock_dict.items():
        stock_predicted = get_predict_data(v, date)
        if stock_predicted is not None:
            predicted_list.append(stock_predicted)
            code_list.append(c)

    if 0 == len(predicted_list):
        return None
    
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(predicted_list)},
      num_epochs=1,
      shuffle=False)
    
    predictions = list(classifier.predict(input_fn=predict_input_fn))
#     predicted_classes = int(predictions[0]["classes"][0])
#     predicted_probability = predictions[0]['probabilities'][predicted_classes]
    # print("  {} {} {} {}".format(code, date, predicted_classes, predicted_probability))
    
#     return predictions, code_list

    result = [[code_list[idx], int(r["classes"][0]), r['probabilities'][int(r["classes"][0])]] for idx, r in enumerate(predictions)]

    result = [x for x in result if x[2] > 0.8]
    
    for x in result:
        k = stock_dict[x[0]]
        si = k.index.get_loc(date)
        ei = sld.get_idx_trend_end_from_st(k, si)
        x.append(sld.get_learning_range_pc(k, si, ei))    
                 
#     cat = [p.insert(0, c) for c, p in zip(code_list, result)]
    predicted_df = pd.DataFrame(result, columns=['code', 'class', 'probability', 'pc']) 
    return predicted_df

In [4]:
def predict_stock_range(classifier, stock_dict, date_start, date_end):
    if date_start is None or date_end is None:
        print("Please provide the date range.")
        return
    
#     for c, v in stock_dict.items():
#         sl.preliminary_data(v)
        
    start = datetime.datetime.strptime(date_start, '%Y/%m/%d')
    end = datetime.datetime.strptime(date_end, '%Y/%m/%d')
    
    one_day = datetime.timedelta(1)
    
    while start <= end:
        weekday = start.isoweekday()
        if 6 == weekday or 7 == weekday:
            start += one_day
            continue
            
        day_for_predicted = start.strftime('%Y/%m/%d')
#         print(day_for_predicted)
        
        df = get_predicted_df(classifier, stock_dict, day_for_predicted)
        start += one_day
        if df is None:
            continue
            
#         raise_df = df[df['class'] >= 4]
#         raise_df = raise_df[raise_df['probability'] > 0.8]
        
#         down_df = df[df['class'] <= 2]
#         down_df = down_df[down_df['probability'] > 0.8]
#         if 0 == len(raise_df) and 0 == len(down_df):
#             continue

#         df = df[df['probability'] > 0.8]
        if 0 == len(df):
            continue
        
        class_one = len(df[df['class'] == 1])
        rate = class_one/len(df)
        print("\n{} {}\t{}".format(day_for_predicted, 'up' if rate > 0.8 else 'fall', rate))
#         print();print("{}\t{}".format(day_for_predicted, class_one/len(df)))
        for idx, row in df.iterrows():
                print("{} {} {} {}".format(row['code'], row['class'], row['probability'], row['pc']))



    
#         for idx, row in raise_df.iterrows():
#             print("{} {} {}".format(row['code'], row['class'], row['probability']))
            
#         for idx, row in down_df.iterrows():
#             print("{} {} {}".format(row['code'], row['class'], row['probability']))

def predict_stock_today(classifier, stock_dict):
    today = datetime.datetime.today()
    today_str = today.strftime('%Y/%m/%d')
    predict_stock_range(classifier, stock_dict, today_str, today_str)

In [5]:
def predict_stock_day(classifier, stock_dict, date_str, code=None):
    if date_str is None:
        print("Please provide the date range.")
        return
    
#     for c, v in stock_dict.items():
#         sld.preliminary_data(v)

    day_for_predicted = date_str
#         print(day_for_predicted)

    df = get_predicted_df(classifier, stock_dict, day_for_predicted)
    if df is None or 0 == len(df):
        return


    class_one = len(df[df['class'] == 1])
    rate = class_one/len(df)
    print("{} {}\t{}".format(day_for_predicted, 'up' if rate > 0.8 else 'fall', rate))
    
    if code is not None:
        df = df[df['code'] == code]

    for idx, row in df.iterrows():
        print("{} {} {} {}".format(row['code'], row['class'], row['probability'], row['pc']))

In [6]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [7]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[180])]
classifier_1024_512_256 = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                                     hidden_units=[1024, 512, 256],
                                                     n_classes=2,
                                                     model_dir="f:/Project/PcAnalyse/model_hu_1024_512_256_nc_2_trend_20")

In [8]:
stock_dict_predict = sdl.load_path2dict('f:/StockData/real', '2017/10/11')

In [9]:
predict_stock_today(classifier_1024_512_256, stock_dict_predict)


2017/11/30 up	0.8062678062678063
002354 1 0.8008437156677246 0.0
600823 1 0.921384334564209 0.0
300446 0 0.8840575218200684 0.0
000735 1 0.9194257855415344 0.0
002609 1 0.9677473306655884 0.0
601099 1 0.8644717931747437 0.0
600650 0 0.8050980567932129 0.0
600354 0 0.9268911480903625 0.0
300284 1 0.884965717792511 0.0
600438 0 0.9284207224845886 0.0
300496 1 0.9048226475715637 0.0
300661 1 0.9932870268821716 0.0
300133 1 0.9615511894226074 0.0
603222 1 0.963939368724823 0.0
002730 0 0.9624818563461304 0.0
002151 1 0.9344252347946167 0.0
000545 1 0.9586498737335205 0.0
000404 1 0.9812151789665222 0.0
300502 0 0.8063113689422607 0.0
300407 0 0.8997326493263245 0.0
600801 1 0.9896061420440674 0.0
600980 0 0.8853287100791931 0.0
600740 1 0.9854459762573242 0.0
600243 1 0.9218082427978516 0.0
002440 1 0.9001693725585938 0.0
600367 1 0.8254344463348389 0.0
002320 1 0.915881872177124 0.0
603008 1 0.8918370008468628 0.0
000666 1 0.9744826555252075 0.0
600031 1 0.9721007347106934 0.0
000779 1 0

300473 1 0.8461136221885681 0.0
603036 1 0.9529005289077759 0.0
000898 1 0.9117035269737244 0.0
002893 1 0.9976158142089844 0.0
600383 1 0.9273098111152649 0.0
600586 1 0.8723618388175964 0.0
002765 0 0.8978068232536316 0.0
002653 0 0.9470478296279907 0.0
000403 0 0.9271231293678284 0.0
601898 1 0.8637276291847229 0.0
000912 1 0.992780864238739 0.0
600123 1 0.9661272168159485 0.0
300029 0 0.9390630722045898 0.0
600081 1 0.803076446056366 0.0
002140 1 0.9486826658248901 0.0
300605 0 0.9291622638702393 0.0
002605 1 0.8334571719169617 0.0
002707 1 0.9244081377983093 0.0
300591 1 0.8021264672279358 0.0
600511 1 0.8174625635147095 0.0
603660 1 0.8869664072990417 0.0
300273 1 0.8535997271537781 0.0
603966 0 0.8451875448226929 0.0
000650 1 0.8951336741447449 0.0
002374 1 0.8405631184577942 0.0
600130 1 0.9365877509117126 0.0
000910 1 0.861899197101593 0.0
002789 1 0.9672248959541321 0.0
603367 1 0.8931618332862854 0.0
603031 0 0.8008003830909729 0.0
000738 1 0.9353217482566833 0.0
300223 1 0.

In [10]:
#predict_stock_day(classifier_1024_512_256, stock_dict_predict, '2017/10/09', '300295')

In [14]:
date_str = datetime.datetime.today().strftime('%Y/%m/%d')
print(date_str)
stock_lst = sbt.backtest_all_stock_specify_date(stock_dict_predict, date_str)

2017/11/30
	002181
	300290
	000836
	000869
	601127
	603303
	000863
	002089
	000532
	600163
	300617
	300437
	002845
	603103
	002467
	300668
	000070
	002877
	600387
	603586
	002785
	300153
	300583
	002766
	300548
	300619
	002448
	300588
	000100
	002445
	600722
	000692
	603887
	300384
	002776
	603617
	300553
	300160
	300559
	002818
	600209
	603367
	603358
	300530
	002788
	603615
	603963
	601798
	603368
	300093
	600099
	603919
	300581
	300447
	000428
	603138
	603139
	300691
	603001
	002758
	300098
	300673
	603901
	603007
	600817
	603366
	600423
	002403
	603298
	600843
	300299
	603823
	002329
	002849
